In [1]:
import tensorflow as tf
from model2c.pykeras import convert
import os

In [2]:
# get data
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0


In [3]:
# define a simple sequential model
def create_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

  return model

model = create_model()
model.summary()


2022-07-15 20:44:12.314167: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-15 20:44:12.358912: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-15 20:44:12.359062: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


2022-07-15 20:44:12.359853: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-15 20:44:12.360742: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-15 20:44:12.360866: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-15 20:44:12.360965: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [4]:
# training
model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))


Epoch 1/10
32/32 [==============================] - 1s 5ms/step - loss: 1.1650 - sparse_categorical_accuracy: 0.6660 - val_loss: 0.7274 - val_sparse_categorical_accuracy: 0.7680
Epoch 2/10
 1/32 [..............................] - ETA: 0s - loss: 0.5666 - sparse_categorical_accuracy: 0.7812

2022-07-15 20:44:13.622887: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


32/32 [==============================] - 0s 2ms/step - loss: 0.4427 - sparse_categorical_accuracy: 0.8740 - val_loss: 0.5247 - val_sparse_categorical_accuracy: 0.8520
Epoch 3/10
32/32 [==============================] - 0s 2ms/step - loss: 0.2881 - sparse_categorical_accuracy: 0.9330 - val_loss: 0.4608 - val_sparse_categorical_accuracy: 0.8560
Epoch 4/10
32/32 [==============================] - 0s 3ms/step - loss: 0.2219 - sparse_categorical_accuracy: 0.9430 - val_loss: 0.4290 - val_sparse_categorical_accuracy: 0.8630
Epoch 5/10
32/32 [==============================] - 0s 3ms/step - loss: 0.1557 - sparse_categorical_accuracy: 0.9620 - val_loss: 0.4532 - val_sparse_categorical_accuracy: 0.8590
Epoch 6/10
32/32 [==============================] - 0s 3ms/step - loss: 0.1293 - sparse_categorical_accuracy: 0.9710 - val_loss: 0.4080 - val_sparse_categorical_accuracy: 0.8650
Epoch 7/10
32/32 [==============================] - 0s 3ms/step - loss: 0.0869 - sparse_categorical_accuracy: 0.9860 - va

In [5]:
# export the model

convert(model, (28, 28, 1), quantization="fp32")
print(f"size of model: {os.path.getsize('model.c')/1024} kilobytes")

INFO:tensorflow:Assets written to: keras_model/assets


/home/hexagon/miniconda3/envs/torch39/lib/python3.9/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2022-07-15 20:44:16,732 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2022-07-15 20:44:16,836 - INFO - Signatures found in model: [serving_default].
2022-07-15 20:44:16,836 - WARNING - '--signature_def' not specified, using first signature: serving_default
2022-07-15 20:44:16,836 - INFO - Output names: ['dense_1']
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2022-07-15 20:44:16,878 - WARNING - From /home/hexagon/miniconda3/envs/torch39/lib/python3.9/site-packages/tf2onnx/tf_loader.py:711: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.e

size of model: 10599.9970703125 kilobytes


2022-07-15 20-44-17.387 [Warn] (getIoTensor) Graph input tensor dimension (unk__6) not specified!
2022-07-15 20-44-17.387 [Warn] (getIoTensor) Specify with command line option '-d unk__6:<value>'
2022-07-15 20-44-17.387 [Warn] (getIoTensor) Defining this dimension as 1 for now.
2022-07-15 20-44-17.388 [Warn] (getIoTensor) Graph input tensor dimension (unk__7) not specified!
2022-07-15 20-44-17.388 [Warn] (getIoTensor) Specify with command line option '-d unk__7:<value>'
2022-07-15 20-44-17.388 [Warn] (getIoTensor) Defining this dimension as 1 for now.
